# Functional Programming in Python
## David Mertz
### dmertz@continuum.io
### 2016-04-22

<hr/>
This tutorial, and Python in general, run more smoothly under Python 3.x.

Whether you're running on Python 2 or Python 3, please install [Python-Future](http://python-future.org/futurize.html):
```bash
conda install future
```

In [ ]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
from future import standard_library
standard_library.install_aliases()
from future.builtins import (
         bytes, dict, int, list, object, range, str,
         ascii, chr, hex, input, next, oct, open,
         pow, round, super, filter, map, zip)

# Table of Contents
* [Container classes](#Container-classes)
* [Class-based iterables](#Class-based-iterables)

# Container classes

If `my_list` is a list, then the expression `my_list[2]` is actually *a call to the method `.__getitem__()` from the `list` class*;  in particualr, `my_list.__getitem__(2) is my_list[2]`. The same is true for other classes of objects that support indexing with brackets (e.g., `tuple`, `dict`, etc.).

In [ ]:
my_list = [0,2,3,4,5,6]
my_list[2] is my_list.__getitem__(2)

In [ ]:
dct = {'David':'Mertz', 'James':'Lowden'}
dct['David'] is dct.__getitem__('David')

In [ ]:
dct.__getitem__('James')

Thus, we can customize the `__getitem__` any way we like to construct unusual classes with arbitrary indexing behavior. For instance, the `Capitalizer` class defined below (as a *subclass* of `dict`) can accept arbitrary strings as indices, even if they are not stored in the underlying dictionary.

In [ ]:
class Capitalizer(dict):
    def __getitem__(self, word):
        return word.upper()

In [ ]:
capitalizer = Capitalizer()
print(capitalizer['Ivan'])

In [ ]:
capitalizer['David'] = 'Mertz' # The assignment of key-value pairs inherits from dict
print(capitalizer['David'])    # However, __getitem__() does not do what a dict does!

In [ ]:
capitalizer['Moise'] = "Smith"
print(capitalizer.keys())
print(capitalizer.items()) # We can examine the key-value pairs as for a dict

As a nifty example, let's define an `ExpandingSequence` class with customized `__getitem__` and `__len__` methods as well as attributes `iteration` and `_cache`. This class is a subclass of the `Sequence` class in the `collections.abc` module.

In [ ]:
# A bit of cleverness to create a new sequence for infinite iterators
from collections.abc import Sequence
class ExpandingSequence(Sequence):
    def __init__(self, iterator):
        self.iterator = iterator
        self._cache = []
    def __getitem__(self, index):
        while len(self._cache) <= index:
            self._cache.append(next(self.iterator))
        return self._cache[index]
    def __len__(self):
        return len(self._cache)

We can build a generator function to build a sequence of arbitrary length using lazy evaluation. For instance, the function below builds a list of primes using the Sieve of Eratosthenes.

In [ ]:
# A generator function to build up a list of prime numbers
def get_primes():
    "Simple lazy Sieve of Eratosthenes"
    candidate = 2
    primes = [] # The list of primes is accumulated 
    while True:
        if all(candidate % prime != 0 for prime in primes):
            yield candidate
            primes.append(candidate)
        candidate += 1

In [ ]:
def fib_generator():
    prev, curr = 0, 1
    while True:
        yield curr
        prev, curr = curr, prev + curr

We can instantiate the class `ExpandingSequence` defined above to generate a sequence of primes that can be accessed as if it were a list that grows on demand.

In [ ]:
primes = ExpandingSequence(get_primes())
for _, p in zip(range(10), primes):
    print(p, end=" ")

In [ ]:
print(primes)

In [ ]:
primes[10] # What is the 11th prime number?

In [ ]:
primes[5] # What is the 6th prime number?

In [ ]:
len(primes) # Up to now, only 11 primes hove been generated

In [ ]:
primes[100] # The ExpandingSequence object now built up to 101 entries

In [ ]:
len(primes)

In [ ]:
print(primes._cache)

In [ ]:
257 in primes._cache

In [ ]:
257 in primes   # Why does this work?

This last code sample is interesting.  The `in` operator has several fallbacks in how it operates.  The first thing the operator tries is using the `.__contains__()` method.  But if that is not available, it will try calling `.__getitem__()` with sequential indices: 0, 1, 2, $...$; this works great when the number is found after a finite number of checks, but for a composite number, it will keep looking infinitely far.

In other words, DO NOT do this with current implementation (it will run forever):

```python
if 258 in primes: 
    print("A composite number was found in primes")
```

We can improve our class though if we want, for this purpose.

In [ ]:
class MonotonicExpandingSequence(ExpandingSequence):
    def __contains__(self, num):
        i = 0
        while True:
            if self[i] == num:
                return True
            elif self[i] > num:
                return False
            else:
                i += 1
                
primes = MonotonicExpandingSequence(get_primes())
257 in primes, 258 in primes

In [ ]:
len(primes), primes

# Class-based iterables

Defining generator functions is almost always more readable and more concise than using a class definition.  However, there may be cases where a custom class is necessary or more clear in defining complex behaviors.  Below is a simple class defining an iterator (and hence an iterable).

In [ ]:
class Letters(object):
    def __init__(self, s):
        self.s = s
        self.pos = -1
        
    def __iter__(self):
        return self
    
    def __next__(self):
        try:
            self.pos += 1
            return self.s[self.pos]
        except IndexError:
            raise StopIteration
            
for letter in Letters("Class Based"):
    print(letter, end="_")

A generator function the returns a sequence of all the primes was presented in the *Introduction to Python* notebook.  While we cannot loop over infinitely many elements, we can still either break within a loop or otherwise limit the looping.

In [ ]:
import continuum_style; continuum_style.style()